## Import packages and utilities 

In [554]:
import pandas as pd
import numpy as np 
import datetime 
from datetime import datetime, date, time
from pandas import ExcelWriter
from pandas import ExcelFile 

### Load the dataset 

In [555]:
# Load the data set 
store_master=pd.read_excel("C:/Users/ssharan/Desktop/GML/Data/Store_Master.xlsx")

customer_transaction = pd.read_excel("C:/Users/ssharan/Desktop/GML/Data/Customer_Transaction.xlsx")

customer_demographics = pd.read_excel("C:/Users/ssharan/Desktop/GML/Data/Customer_Demographics.xlsx")

test_set = pd.read_excel("C:/Users/ssharan/Desktop/GML/Data/Test_Set.xlsx")



In [556]:
print(store_master.shape)
print(customer_transaction.shape)
print(customer_demographics.shape)
print(test_set.shape)

(35, 16)
(544649, 15)
(100000, 16)
(400000, 2)


#### Customer Transaction Data Cleaning

In [557]:
# Drop Columns
customer_transaction.drop(['Territory','Business','Return_Reason'],axis=1,inplace=True)

In [558]:
customer_transaction.head()

,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Customer_ID,Invoices,Item_Count,Revenue,Discount,Units_Sold
0,2010,0,60065,Dubai,Stand alone,Return,1800000058056860,1,3,-169.0,0.0,-3
1,2016,0,60053,NaN,NaN,Purchase,1800000020917140,1,1,0.0,0.0,0
2,2016,0,60053,NaN,NaN,Purchase,1800000021265010,1,1,0.0,0.0,0
3,2016,0,60053,NaN,NaN,Purchase,1800000025693950,1,4,0.0,0.0,0
4,2016,0,60053,NaN,NaN,Purchase,1800000044104620,1,1,0.0,0.0,0


In [91]:
table1=  pd.DataFrame(customer_transaction['Customer_ID'].value_counts(), columns=['Customer_ID'])
table1.reset_index(inplace=True)
table1.rename(columns={'index':'Customer_ID','Customer_ID':'Prediction'},inplace=True) 

#### Customer Demographics Data Cleaning 

In [559]:
customer_demographics[['Birth_date', 'First_txn_dt','Last_accr_txn_dt','Last_rdm_txn_dt']] = customer_demographics[['Birth_date', 'First_txn_dt','Last_accr_txn_dt','Last_rdm_txn_dt']].astype(str)
customer_demographics['First_txn_dt'].fillna(customer_demographics['First_txn_dt'].mode()[0])
customer_demographics['Last_accr_txn_dt'].fillna(customer_demographics['Last_accr_txn_dt'].mode()[0])

customer_demographics['Birth_date']=pd.to_datetime(customer_demographics['Birth_date'],format='%d%b%Y:%H:%M:%S')
customer_demographics['First_txn_dt']=pd.to_datetime(customer_demographics['First_txn_dt'],format='%d%b%Y:%H:%M:%S')
customer_demographics['Last_accr_txn_dt']=pd.to_datetime(customer_demographics['Last_accr_txn_dt'],format='%d%b%Y:%H:%M:%S')
customer_demographics['Last_rdm_txn_dt']=pd.to_datetime(customer_demographics['Last_rdm_txn_dt'],format='%d%b%Y:%H:%M:%S')

# Drop Columns
customer_demographics.drop(['Territory'],axis=1,inplace=True)

customer_demographics['Points'].fillna(customer_demographics['Points'].mean(),inplace=True)

customer_demographics['Engaged_years']= (customer_demographics['Last_accr_txn_dt'] -customer_demographics['First_txn_dt'] ).astype('<m8[Y]')



In [561]:
customer_demographics.head()


,Customer_ID,Nationality,Income_Range,Job_Type,Marital_Status,Gender,State,Language,Loyalty_Status,Birth_date,Age,Points,First_txn_dt,Last_accr_txn_dt,Last_rdm_txn_dt,Engaged_years
0,1800000006365760,INDIA,Below 5000,Services,Married,M,Abu Dhabi,English,Gold,1959-06-01,59.0,814.0,2010-11-02,2018-09-27 21:45:25,2018-03-11 19:11:57,7.0
1,1800000006365820,UNITED ARAB EMIRATES,Below 5000,Services,Married,F,Abu Dhabi,Arabic,Silver,1900-01-01,NaN,212.0,2010-12-06,2016-08-16 21:08:28,2015-03-16 13:39:12,5.0
2,1800000006366060,SRI LANKA,Below 5000,Services,Married,M,Abu Dhabi,English,Silver,1959-06-07,59.0,186.0,2011-02-17,2018-08-31 20:52:18,2017-01-30 19:19:19,7.0
3,1800000006366230,JORDAN,Below 5000,Services,Married,F,Abu Dhabi,Arabic,Silver,1962-01-21,57.0,126.0,2010-10-30,2018-07-30 20:30:05,2017-06-20 17:57:27,7.0
4,1800000006366570,OCCUPIED PALESTINIAN TERRITORY,Below 5000,Services,Married,M,Abu Dhabi,Arabic,Silver,1900-01-01,NaN,197.0,2010-11-10,2018-09-03 22:10:56,2017-06-17 23:08:28,7.0


### Store Master Feature Engineering

In [562]:
from math import sin, cos, sqrt, atan2, radians


def distance(dlat,dlon):
    R=6373 # approximate radius of earth in km
    a=sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance= R*c
    return(distance)


In [563]:
store_master['Store_Format']= store_master['Store_Format'].astype('category')
store_master['Region']=store_master['Region'].astype('category')
store_master['Region_Code']=store_master['Region_Code'].astype('category')
store_master['Store_Format'] = np.where(store_master['Mall_Name']=='STANDALONE',store_master['Mall_Name'], store_master['Store_Format'])
store_master['Store_Format']=['Stand Alone Stores' if x == 'STANDALONE' else x for x in store_master['Store_Format']]
store_master['Store_Format']=['Mall Stores' if x is np.nan else x for x in store_master['Store_Format']]

# Drop Columns 
store_master.drop(["Territory", "Business","Store_Name","Mall_Name"],axis=1,inplace=True)


In [564]:

store_master['Longitude'] = store_master.apply(lambda row: radians(row['Longitude']), axis=1)
store_master['Latitude'] = store_master.apply(lambda row: radians(row['Latitude']), axis=1)
store_master['ARPU']= store_master['Total_Revenue']/store_master['Customer_Count']
store_master['TRPSF']=store_master['Total_Revenue']/store_master['Store_Size_Sq_Ft']
store_master['Store_Age'] =(pd.Timestamp(datetime.now()) - store_master['Store_Launch_Date']).astype('<m8[Y]')

store_master['Distance_60301']= store_master.apply(lambda row: distance(store_master.loc[store_master['Train_Test_Store']=='Test',['Latitude']].values[0]-row['Latitude'], store_master.loc[store_master['Train_Test_Store']=='Test',['Longitude']].values[0]-row['Longitude']), axis=1)
store_master['Distance_60298']= store_master.apply(lambda row: distance(store_master.loc[store_master['Train_Test_Store']=='Test',['Latitude']].values[1]-row['Latitude'], store_master.loc[store_master['Train_Test_Store']=='Test',['Longitude']].values[1]-row['Longitude']), axis=1)
store_master['Distance_60300']= store_master.apply(lambda row: distance(store_master.loc[store_master['Train_Test_Store']=='Test',['Latitude']].values[2]-row['Latitude'], store_master.loc[store_master['Train_Test_Store']=='Test',['Longitude']].values[2]-row['Longitude']), axis=1)
store_master['Distance_60299']= store_master.apply(lambda row: distance(store_master.loc[store_master['Train_Test_Store']=='Test',['Latitude']].values[3]-row['Latitude'], store_master.loc[store_master['Train_Test_Store']=='Test',['Longitude']].values[3]-row['Longitude']), axis=1)

In [565]:
store_master.nsmallest(8,'Distance_60298')
store_master.nsmallest(8,'Distance_60299')
store_master.nsmallest(8,'Distance_60300')
store_master.nsmallest(8,'Distance_60301')


,Region,Region_Code,Store_Code,Store_Format,Sales_Per_Day,Store_Launch_Date,Train_Test_Store,Longitude,Latitude,Store_Size_Sq_Ft,Customer_Count,Total_Revenue,ARPU,TRPSF,Store_Age,Distance_60301,Distance_60298,Distance_60300,Distance_60299
31,MX - Abu Dhabi,602,60301,Mall Stores,10861.009520,2018-01-30,Test,0.427092,0.948982,23288.338616,NaN,7.472375e+06,NaN,320.863359,0.0,0.000000,135.226796,2.015630,157.542545
28,MX - Abu Dhabi,602,60053,Stand Alone Stores,3094.424495,2004-05-16,Train,0.427502,0.948888,19200.122169,160218.0,6.436403e+07,401.727830,3352.271872,14.0,1.635326,135.185619,1.162052,158.301448
33,MX - Abu Dhabi,602,60300,Mall Stores,7036.233391,2018-02-01,Test,0.427390,0.948718,6604.762130,NaN,2.701914e+06,NaN,409.085682,0.0,2.015630,136.347195,0.000000,159.334611
30,MX - Abu Dhabi,602,60275,Mall Stores,498.182287,2017-03-20,Train,0.427658,0.949161,2178.431368,13669.0,5.271267e+06,385.636607,2419.753433,1.0,2.390537,133.355255,2.996757,156.636871
18,MX - Abu Dhabi,602,60069,Mall Stores,1813.094697,2011-07-30,Train,0.426466,0.949679,6698.117964,114246.0,3.771237e+07,330.097944,5630.293449,7.0,5.011502,132.125534,7.020135,152.899261
9,MX - Abu Dhabi,602,60067,Mall Stores,1699.560982,2011-12-14,Train,0.427190,0.948088,8673.534535,89351.0,3.535087e+07,395.640434,4075.716571,7.0,5.710948,140.341474,4.082504,163.249344
29,MX - Abu Dhabi,602,60054,Mall Stores,2101.072259,2011-11-23,Train,0.424700,0.951623,12425.516106,118444.0,4.370230e+07,368.970171,3517.141873,7.0,19.025927,124.173902,21.033306,140.077250
23,MX - Abu Dhabi,602,60233,Mall Stores,1045.779749,2014-11-25,Train,0.427406,0.953105,16455.386430,75505.0,2.175222e+07,288.089779,1321.890487,4.0,26.298891,111.029776,27.958345,131.582025


##### Rules
###### 60298 nearest stores are 60055,(60075 9 kms)
###### 60299 nearest store are 60057
######  60300 nearest stores are 60053,60301,60275,60067,60069
######  60301 nearest stores are 60053,60300,60275,60067,60069

In [566]:
store_master.head()

,Region,Region_Code,Store_Code,Store_Format,Sales_Per_Day,Store_Launch_Date,Train_Test_Store,Longitude,Latitude,Store_Size_Sq_Ft,Customer_Count,Total_Revenue,ARPU,TRPSF,Store_Age,Distance_60301,Distance_60298,Distance_60300,Distance_60299
0,MX - Abu Dhabi,602,60077,Mall Stores,544.074849,2011-11-02,Train,0.447121,0.932733,4852.220341,22237.0,1.131676e+07,508.915630,2332.284205,7.0,127.482333,228.701004,125.468474,275.547750
1,MX - Sharjah,606,60073,Mall Stores,1251.656549,2009-08-21,Train,0.441743,0.966538,13789.949661,88372.0,2.603446e+07,294.600736,1887.929750,9.0,124.399177,12.338898,125.443256,83.535008
2,MX - Dubai,601,60065,Mall Stores,3319.885702,2005-04-15,Train,0.437112,0.962030,15609.687991,188836.0,6.905362e+07,365.680393,4423.767000,13.0,91.093736,44.159193,92.196099,90.840060
3,MX - Dubai,601,60219,Stand Alone Stores,1211.477261,2014-06-12,Train,0.440146,0.966202,6845.497058,96733.0,2.519873e+07,260.497731,3681.066083,4.0,119.963613,15.305705,121.070329,79.837243
4,MX - Ras Al Khaimah,605,60071,Stand Alone Stores,2221.872874,2006-03-20,Train,0.450090,0.976825,12385.047461,70456.0,4.621496e+07,655.940669,3731.512206,12.0,196.910270,62.054405,197.956915,103.559536


### Wrangling 

In [567]:
table2 =  pd.merge(customer_transaction,store_master,how='left',on='Store_Code')

In [636]:
table2.pivot_table(index='Store_Code',columns='Region_Code',values='Customer_ID',aggfunc='count',fill_value=0)


Region_Code,601,602,603,604,605,606
Store_Code,,,,,,
60053,0,86280,0,0,0,0
60054,0,26416,0,0,0,0
60055,0,0,10651,0,0,0
60057,0,24454,0,0,0,0
60058,0,15544,0,0,0,0
60059,11728,0,0,0,0,0
60060,8338,0,0,0,0,0
60061,17764,0,0,0,0,0
60062,6207,0,0,0,0,0


In [568]:
table2['Store_Code1']=table2['Store_Code']

In [569]:
table2.shape

(544649, 31)

In [757]:
pivot1 = table2.pivot_table(index='Customer_ID',columns='Region_Code',values='Store_Code',aggfunc='count',fill_value=0)
pivot1=pivot1.astype(int)
pivot1.head()

Region_Code,601,602,603,604,605,606
Customer_ID,,,,,,
1800000006365760,0,4,0,0,0,0
1800000006365820,0,1,0,0,0,0
1800000006366060,1,5,0,0,0,0
1800000006366230,0,6,0,0,0,0
1800000006366570,0,10,0,0,0,0


In [758]:
pivot1.shape

(100000, 6)

In [1009]:
pivot2 = table2.pivot_table(index='Customer_ID',columns='Store_Code',values='Store_Code1',aggfunc='count',fill_value=0)

In [1010]:
pivot2.head()

#60298 nearest stores are 60055,(60075 9 kms)
#60299 nearest store are 60057
#60300 nearest stores are 60053,60301,60275,60067,60069
#60301 nearest stores are 60053,60300,60275,60067,60069

Store_Code,60053,60054,60055,60057,60058,60059,60060,60061,60062,60063,...,60076,60077,60079,60084,60086,60219,60233,60241,60250,60275
Customer_ID,,,,,,,,,,,,,,,,,,,,,
1800000006365760,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1800000006365820,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1800000006366060,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,2,0,0,0
1800000006366230,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1800000006366570,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1011]:
pivot2.loc[((pivot2[60057]>1) & (pivot2[60069]>1)),].count()

Store_Code
60053    114
60054    114
60055    114
60057    114
60058    114
60059    114
60060    114
60061    114
60062    114
60063    114
60064    114
60065    114
60066    114
60067    114
60068    114
60069    114
60070    114
60071    114
60072    114
60073    114
60074    114
60075    114
60076    114
60077    114
60079    114
60084    114
60086    114
60219    114
60233    114
60241    114
60250    114
60275    114
dtype: int64

## Implementing Rules

In [1012]:
conditions_298 = [((pivot2[60055]>=1) & (pivot2[60075]>=1)),((pivot2[60055]==1) | (pivot2[60075]>0)) ]
choices_298 = [1,1]
pivot2['60298'] = np.select(conditions_298,choices_298,default=0)

conditions_299 = [(pivot2[60057]>1)]
choices_299 = [1]
pivot2['60299'] = np.select(conditions_299,choices_299,default=0)

conditions_300 = [((pivot2[60053]>0) | (pivot2[60275]>0) | (pivot2[60067]>0)),((pivot2[60067]>1) & (pivot2[60069]>1)), (pivot2[60069]>1) ]
choices_300 = [1,1,1]
pivot2['60300'] = np.select(conditions_300,choices_300,default=0)

conditions_301 = [((pivot2[60053]>0) | (pivot2[60275]>0) | (pivot2[60067]>0)),((pivot2[60067]>0) & (pivot2[60069]>0)), (pivot2[60069]>0) ]
choices_301 = [1,1,1]
pivot2['60301'] = np.select(conditions_301,choices_301,default=0)

In [1013]:
# pivot2.head()
pivot2.shape

(100000, 36)

In [1014]:
col_list= list(pivot2)
col_list.remove('60298')
col_list.remove('60299')
col_list.remove('60300')
col_list.remove('60301')


In [1015]:
pivot2['not_Store_visited'] = pivot2[col_list].sum(axis=1)

In [1016]:
table3=pd.merge(test_set,pivot2,how='left',on='Customer_ID')

In [1017]:
table4=table3[['Customer_ID','Store_Code','60298','60300','60299','60301']]
table4.head()
table4['Prediction'] = 0

C:\Users\ssharan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1018]:
table4.loc[(table4['Store_Code']==60298) & (table4['60298']==1) ,['Prediction']]=1
table4.loc[(table4['Store_Code']==60299) & (table4['60299']==1),['Prediction']]=1
table4.loc[(table4['Store_Code']==60300) & (table4['60300']==1),['Prediction']]=1
table4.loc[(table4['Store_Code']==60301) & (table4['60301']==1),['Prediction']]=1

C:\Users\ssharan\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [1019]:
table4.head()

,Customer_ID,Store_Code,60298,60300,60299,60301,Prediction
0,1800000006365760,60298,0,1,0,1,0
1,1800000006365760,60299,0,1,0,1,0
2,1800000006365760,60300,0,1,0,1,1
3,1800000006365760,60301,0,1,0,1,1
4,1800000006365820,60298,0,0,0,0,0


In [1020]:
new_pivot2=pd.merge(table4,customer_demographics,how='left',on='Customer_ID')
new_pivot2=pd.merge(new_pivot2,pivot2[['not_Store_visited',60076,60075]],how='left',on='Customer_ID')
new_pivot2=pd.merge(new_pivot2,pivot1[[601.0,602.0,603.0,604.0,605.0,606.0]],how='left',on='Customer_ID')


In [1021]:
new_pivot2['Last_transaction_year']= pd.DatetimeIndex(new_pivot2['Last_accr_txn_dt']).year
new_pivot2['Last_transaction_month']= pd.DatetimeIndex(new_pivot2['Last_accr_txn_dt']).month

new_pivot2['Cat_points']= np.where(new_pivot2['Points']< 450, 0, 1)
new_pivot2['Match_year']= np.where(((pd.DatetimeIndex(new_pivot2['Last_accr_txn_dt']).year)==(pd.DatetimeIndex(new_pivot2['First_txn_dt']).year)), 1, 0)
new_pivot2['Match_month']= np.where(((pd.DatetimeIndex(new_pivot2['Last_accr_txn_dt']).month)==(pd.DatetimeIndex(new_pivot2['First_txn_dt']).month)), 1, 0)

In [1022]:
new_pivot2.head()

,Customer_ID,Store_Code,60298,60300,60299,60301,Prediction,Nationality,Income_Range,Job_Type,...,602,603,604,605,606,Last_transaction_year,Last_transaction_month,Cat_points,Match_year,Match_month
0,1800000006365760,60298,0,1,0,1,0,INDIA,Below 5000,Services,...,4,0,0,0,0,2018.0,9.0,1,0,0
1,1800000006365760,60299,0,1,0,1,0,INDIA,Below 5000,Services,...,4,0,0,0,0,2018.0,9.0,1,0,0
2,1800000006365760,60300,0,1,0,1,1,INDIA,Below 5000,Services,...,4,0,0,0,0,2018.0,9.0,1,0,0
3,1800000006365760,60301,0,1,0,1,1,INDIA,Below 5000,Services,...,4,0,0,0,0,2018.0,9.0,1,0,0
4,1800000006365820,60298,0,0,0,0,0,UNITED ARAB EMIRATES,Below 5000,Services,...,1,0,0,0,0,2016.0,8.0,0,0,0


In [1023]:
# Deleteing the Onetimers F1-score = 49.74
new_pivot2.loc[ ((new_pivot2['Match_year']==1)  & (new_pivot2['not_Store_visited']==1) & (new_pivot2['Last_transaction_year']<2017) & (new_pivot2['Cat_points']==0)  & (new_pivot2['Prediction']==1) ),['Prediction'] ]=0


In [1024]:
new_pivot2.loc[ ((new_pivot2['Match_year']==1)  & (new_pivot2['Last_transaction_year']<2017) & (new_pivot2['not_Store_visited']==2) & ((new_pivot2[602]==2)|(new_pivot2[603]==2)|(new_pivot2[604]==2)|(new_pivot2[605]==2)|(new_pivot2[606]==2) ) & (new_pivot2['Cat_points']==0) & (new_pivot2['Prediction']==1) ),['Prediction'] ]=0
new_pivot2.loc[ ((new_pivot2['Match_year']==1)  & (new_pivot2['Last_transaction_year']<2017) & (new_pivot2['not_Store_visited']==3) & ((new_pivot2[602]==3)|(new_pivot2[603]==3)|(new_pivot2[604]==3)|(new_pivot2[605]==3)|(new_pivot2[606]==3) ) &(new_pivot2['Cat_points']==0) & (new_pivot2['Prediction']==1) ),['Prediction'] ]=0
new_pivot2.loc[ ((new_pivot2['Match_year']==1)   & (new_pivot2['Last_transaction_year']<2017) & (new_pivot2['not_Store_visited']==4) & ((new_pivot2[602]==4)|(new_pivot2[603]==4)|(new_pivot2[604]==4)|(new_pivot2[605]==4)|(new_pivot2[606]==4) ) & (new_pivot2['Cat_points']==0) & (new_pivot2['Prediction']==1) ),['Prediction'] ]=0



In [1025]:
 new_pivot2.loc[ ( (new_pivot2['Last_transaction_year']==2018) & (new_pivot2['not_Store_visited']>2)  & (new_pivot2['Cat_points']==1) &(new_pivot2[60076]>2) & (new_pivot2['Store_Code']==60298) &(new_pivot2['Prediction']==0) ),['Prediction'] ]=1
# new_pivot2.loc[ ( (new_pivot2['Last_transaction_year']==2018) & (new_pivot2['not_Store_visited']>2)  & (new_pivot2['Cat_points']==1) &(new_pivot2[60075]>4) & (new_pivot2['Store_Code']==60298) &(new_pivot2['Prediction']==0) ),['Prediction'] ]=1


In [1026]:
# new_pivot2.to_csv('C:/Users/ssharan/Desktop/GML/data_view.csv', sep=',', index=None)

In [1027]:
submission1 = new_pivot2[['Customer_ID','Store_Code','Prediction']]
submission1.to_csv('C:/Users/ssharan/Desktop/GML/submission1.csv', sep=',' , index= None)